In [44]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [13]:
sequence_lenght = 4
batch_size = 1
input_dim = 512
d_model = 512

x  = torch.randn((batch_size, sequence_lenght, input_dim))

In [14]:
print(x)

tensor([[[-0.1449, -0.0949, -0.0576,  ..., -1.5725, -0.5864, -1.0207],
         [-0.0747, -0.1532, -0.7710,  ...,  0.5117,  0.2168, -1.4534],
         [-0.3632, -0.6418,  1.3208,  ...,  1.3794,  0.8318, -1.1344],
         [ 1.4652, -0.3772, -0.3034,  ..., -0.1824,  0.3547, -1.0742]]])


In [15]:
qkv_layer = nn.Linear(input_dim, 3 * d_model) 
qkv = qkv_layer(x)
print(qkv.size())

torch.Size([1, 4, 1536])


In [16]:
num_heads = 8
head_dim = d_model // num_heads

qkv = qkv.reshape(batch_size, sequence_lenght, num_heads, 3 * head_dim)

In [17]:
qkv.shape

torch.Size([1, 4, 8, 192])

In [18]:
qkv =  qkv.permute(0,2,1,3)
qkv.shape

torch.Size([1, 8, 4, 192])

In [19]:
q, k, v = qkv.chunk(3, dim=-1)
q.shape, k.shape, v.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

In [39]:
dk = q.size()[-1]
scaled = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(dk)
print(scaled.shape)

torch.Size([1, 8, 4, 4])


In [40]:
mask = torch.full(scaled.size(), float('-inf'))
mask = torch.triu(mask, diagonal= 1)
mask[0][0]

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [41]:
(scaled+mask)[0][0]


tensor([[ 0.5933,    -inf,    -inf,    -inf],
        [-0.0485, -0.4235,    -inf,    -inf],
        [ 0.2481,  0.3452,  0.0223,    -inf],
        [ 0.0911, -0.3600, -0.3852, -0.0432]], grad_fn=<SelectBackward0>)

In [42]:
scaled += mask

In [46]:
attention = F.softmax(scaled, dim=-1)
attention.shape

torch.Size([1, 8, 4, 4])

In [47]:
values = torch.matmul(attention,v)
values.shape

torch.Size([1, 8, 4, 64])

In [ ]:
import math

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [ ]:
values, attention = scaled_dot_product(q, k, v, mask=mask)
attention.shape

In [ ]:
values.size()


In [ ]:
values = values.reshape(batch_size, sequence_lenght, num_heads * head_dim)
values.size()

In [ ]:
linear_layer = nn.Linear(d_model, d_model)
out = linear_layer(values)
out.shape

In [ ]:
out